<a href="https://colab.research.google.com/github/Wingie/bloomon-q/blob/master/bloomon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/code-challange.zip -d /content/


Archive:  /content/code-challange.zip
   creating: /content/code-challange/
  inflating: /content/code-challange/requirements.pdf  
   creating: /content/__MACOSX/
   creating: /content/__MACOSX/code-challange/
  inflating: /content/__MACOSX/code-challange/._requirements.pdf  
  inflating: /content/code-challange/sample.txt  
  inflating: /content/__MACOSX/code-challange/._sample.txt  


In [ ]:
def read_sample(file="/content/code-challange/sample.txt"):
  with open(file) as f:
    return [r.strip() for r in f.readlines()]

sample = read_sample()
blank = sample.index('')
designs = sample[:blank]
flower_stream = sample[blank+1:]
print(designs,len(designs),flower_stream[:10])

['AL10a15b5c30', 'AS10a10b25', 'BL15b1c21', 'BS10b5c16', 'CL20a15c45', 'DL20b28'] 6 ['cL', 'bL', 'aS', 'aS', 'cL', 'cL', 'cS', 'aS', 'cS', 'aS']


In [ ]:
import re
import collections,itertools

class FlowerCounter(collections.Counter):
  def subtract(self, other=None, **kwds):
    "original subtract function didn't handle negatives properly"
    remaining_keys = ([i  for i in other.keys() if i not in self.keys()])
    res = FlowerCounter()
    for k in self:
      res[k] = (self[k] - other[k])
      for k in remaining_keys:
        res[k] = -other[k]
    return res
  def __str__(self):
    return str(sorted(list(self.items())))

# mini test case \o\
small_pile = FlowerCounter(a=1,b=3,c=16,f=9) 
large_pile = FlowerCounter(a=2,b=3,c=9,k=30) 
print(large_pile.subtract(small_pile))

[('a', 1), ('b', 0), ('c', -7), ('f', -9), ('k', 30)]


In [ ]:
def parse_design(design='AL10a15b5c30'):
  b_name = design[0]
  b_size = design[1]
  res = re.findall(r'(\d+)(\w+?)', design[2:])
  b_totalflowers = int(''.join(res[-1]))
  del res[-1] # bad approach. might not scale but ok for purpose of interview test
  flowers = FlowerCounter()
  for f in res:
    flowers[f[1]] = int(f[0])
    
  if(sum(flowers.values()) != b_totalflowers): #sly bloomo devs
    remaining = (b_totalflowers - sum(flowers.values()))
    flowers['*'] = remaining

  return (b_name,b_size,flowers)

b_designs = [parse_design(d) for d in designs]
print(b_designs)


[('A', 'L', FlowerCounter({'b': 15, 'a': 10, 'c': 5})), ('A', 'S', FlowerCounter({'a': 10, 'b': 10, '*': 5})), ('B', 'L', FlowerCounter({'b': 15, '*': 5, 'c': 1})), ('B', 'S', FlowerCounter({'b': 10, 'c': 5, '*': 1})), ('C', 'L', FlowerCounter({'a': 20, 'c': 15, '*': 10})), ('D', 'L', FlowerCounter({'b': 20, '*': 8}))]


In [ ]:

def process_flower_bunch(bunch=[],pile={}):
  # we will process a bunch of flowers at a time, 
  # if we give a bundle with bunch_size=1 then  
  # this fn will process 1 flower at a time
  # print(bunch)
  for flwr in bunch:
    pile[flwr[1]][flwr[0]] += 1

  # print('*start pile is*',pile)
  state = {}
  keep_running = True
  while keep_running:
    for des in b_designs:
        key = des[1]
        if(will_design_fail(des,pile[key])):
          #skip as design not possible
          state[des[0]] = False
          if(not any(state.values()) and len([p for p in pile[key].values() if p < 10]) != 0):
            # no designs were possible and a flower type is below 10 so we bail out on this bunch
            keep_running = False
        else:
          before = FlowerCounter(pile[key])
          # print('pile-before',before)
          pile[key] = process_design(des,pile[key])
          after = FlowerCounter(pile[key])
          print(get_bouquet_str(des,(before.subtract(after))))
          state[des[0]] = True      
  return pile


def get_bouquet_str(des,bouq):
  flwrs = "".join(["{0}{1}".format(d,bouq[d]) for d in sorted(bouq) if bouq[d]!=0])
  return "{0}{1}{2}".format(des[0],des[1],flwrs)

def will_design_fail(des,pile):
  res = pile.subtract(des[2])
  # print(des,pile)
  if('*' in res.keys()):
    cant_make = any([res[r] for r in res if r!='*' and res[r]<0])
  else:
    cant_make = any([res[r] for r in res if res[r]<0])
  return cant_make

def process_design(des,pile):
  # print('1>>>',pile)
  cant_make = will_design_fail(des,pile)
  if cant_make:
    return pile
  res = pile.subtract(des[2])
  if('*' in res.keys() and not cant_make):
    max_flower = pile.most_common(1)[0][0]
    amt =  abs(res['*'])
    if(res[max_flower] > amt):
      res[max_flower] -= amt
      res['*'] = 0
  # print('4>>>',res,des)
  return res



In [ ]:
## actuallu chunk and process file
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

pile = {'S':FlowerCounter(),'L':FlowerCounter()}
for flower_bunch in chunks(flower_stream,2000):
  pile = process_flower_bunch(flower_bunch,pile)
# print('*end  pile*',pile)

ALa10b15c5
ASa15b10
BLa5b15c1
BSa1b10c5
CLa30c15
DLa8b20
ALa10b15c5
ASa15b10
BLa5b15c1
BSa1b10c5
CLa30c15
DLa8b20
ALa10b15c5
ASa15b10
BSa1b10c5
CLa30c15
ASa15b10
BSa1b10c5
CLa30c15
ASa15b10
BSa1b10c5
CLa30c15
ASa15b10
BSa1b10c5
CLa30c15
ASa15b10
BSa1b10c5
ASa15b10
